# Проект "Сервис для предсказания стоимости домов на основе истории предложений"

"Агентство недвижимости" 

К нам обратился представитель крупного агентства недвижимости со
следующей проблемой:
«Мои риелторы тратят катастрофически много времени на сортировку
объявлений и поиск выгодных предложений. Поэтому их скорость реакции, да
и, сказать по правде, качество анализа не дотягивают до уровня конкурентов.
Это сказывается на наших финансовых показателях.

Наша задача — разработать модель, которая позволила бы обойти
конкурентов по скорости и качеству совершения сделок. Вот датасет для
работы».

Что необходимо сделать: разработать сервис для предсказания стоимости
домов на основе истории предложений.

In [4]:
# Импорт библиотек
import os
import sys
import warnings

# ==========================================
# 1. CPU ACCELERATION (Intel)
# ==========================================
try:
    from sklearnex import patch_sklearn
    patch_sklearn()
    print('Intel Extension (CPU) активирован для sklearn')
except (ImportError, Exception):
    print('Intel Extension не найден, работаем на стандартном CPU')

# ==========================================
# 2. GPU ACCELERATION (NVIDIA RAPIDS)
# ==========================================
USE_GPU = False

# Попытка включить "Магический Pandas" (работает только в Jupyter)
try:
    from IPython.core.getipython import get_ipython
    if get_ipython() is not None:
        get_ipython().run_line_magic('load_ext', 'cudf.pandas')
        print("Magic Mode: Pandas автоматически ускорен на GPU!")
        USE_GPU = True
except (ImportError, ModuleNotFoundError):
    pass # Мы не в Jupyter или нет cudf.pandas

import pandas as pd
import numpy as np

# Если магия не сработала (мы в скрипте .py), пробуем импортировать cudf явно
if not USE_GPU:
    try:
        import cudf
        print("Explicit Mode: Библиотека cuDF доступна (используйте cudf вместо pd)")
        USE_GPU = True
    except ImportError:
        print("GPU не найден, работаем на чистом CPU")

# Импорт ML-библиотек (cuml)
try:
    import cuml
    print("cuML (GPU Machine Learning) доступен")
except ImportError:
    pass

from IPython.display import display, Markdown
from matplotlib import pyplot as plt
from tqdm import tqdm

if USE_GPU:
    # Импортируем GPU-версии
    from cuml.ensemble import RandomForestClassifier
    from cuml.linear_model import LinearRegression, LogisticRegression
    from cuml.neighbors import KNeighborsClassifier
    from cuml.svm import SVC
    # Можно добавить любые другие модели, которые есть в cuML    
    print("Модели: загружены из cuML (GPU)")    
else:
    # Импортируем стандартные CPU-версии
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LinearRegression, LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC    
    print("Модели: загружены из Scikit-learn (CPU)")


from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.linalg import svd
from scipy.optimize import minimize, least_squares

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'
from plotly.subplots import make_subplots


import ast
import datetime

from dmba import AIC_score, stepwise_selection
from catboost import CatBoostRegressor
import xgboost as xgb
import optuna
import pickle
import joblib

import scipy
import math

import time


Intel Extension (CPU) активирован для sklearn
The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas
Magic Mode: Pandas автоматически ускорен на GPU!
cuML (GPU Machine Learning) доступен
Модели: загружены из cuML (GPU)


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [6]:
import warnings
warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")

SEED = 42

In [7]:
data = pd.read_csv('./data/data.csv')
data.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,None,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,None,None,None,611019,"$418,000"
1,for sale,None,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",None,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,None,None,201916904,"$310,000"
2,for sale,None,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,None,yes,FR19221027,"$2,895,000"
3,for sale,None,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,None,None,14191809,"$2,395,000"
4,for sale,None,lot/land,1524 Kiscoe St,None,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",None,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",None,32908,None,FL,None,None,None,861745,"$5,000"
